<a href="https://colab.research.google.com/github/dominikblaszczykiewicz/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install hyperopt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

import os, datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt

from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/dw_matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/dw_matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim ==1: y_train = to_categorical(y_train)
if y_test.ndim ==1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs',5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data',(X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(units = 1024, activation='relu'),  
    Dropout(0.3),

    Dense(units = 1024, activation='relu'),  
    Dropout(0.3),

    Dense(units = num_classes, activation='softmax'),                
  ])

In [26]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 22ms/step - loss: 2.4262 - accuracy: 0.3219 - val_loss: 0.6585 - val_accuracy: 0.7973
Epoch 2/5
272/272 [==============================] - 6s 21ms/step - loss: 0.4867 - accuracy: 0.8477 - val_loss: 0.1337 - val_accuracy: 0.9609
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2181 - accuracy: 0.9342 - val_loss: 0.0568 - val_accuracy: 0.9845
Epoch 4/5
272/272 [==============================] - 6s 21ms/step - loss: 0.1444 - accuracy: 0.9577 - val_loss: 0.0459 - val_accuracy: 0.9867
Epoch 5/5
272/272 [==============================] - 6s 21ms/step - loss: 0.1116 - accuracy: 0.9690 - val_loss: 0.0270 - val_accuracy: 0.9924


0.9643990929705215

In [27]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 4ms/step - loss: 0.1711 - accuracy: 0.9644


0.9643990993499756

In [0]:
def get_model(params):
  return Sequential([
                     
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_1']),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_2']),
    
    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_3']),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_1']),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_2']),
    
    Dense(num_classes, activation='softmax')

  ])

In [0]:
def func_obj(params):
  get_model(params)
  model.compile(loss='categorical_crossentropy', 
                optimizer='Adam', 
                metrics=['accuracy']
                )
  
  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)), 
      # epochs=7,
      epochs = int(params.get('epochs', 7)),   
      verbose=0,          
  )

  score = model.evaluate(X_test, y_test,verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [30]:
space = {
      'batch_size': hp.quniform('batch_size', 100, 200, 10),
      'dropout_cnn_block_1': hp.uniform('dropout_cnn_block_1', 0.3, 0.5),
      'dropout_cnn_block_2': hp.uniform('dropout_cnn_block_2', 0.3, 0.5),
      'dropout_cnn_block_3': hp.uniform('dropout_cnn_block_3', 0.3, 0.5),
      'dropout_dense_block_1': hp.uniform('dropout_dense_block_1', 0.3, 0.7),
      'dropout_dense_block_2': hp.uniform('dropout_dense_block_2', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)x


{'batch_size': 130.0, 'dropout_cnn_block_1': 0.30886072061052505, 'dropout_cnn_block_2': 0.46137901497509193, 'dropout_cnn_block_3': 0.49841680621886875, 'dropout_dense_block_1': 0.6835240694931359, 'dropout_dense_block_2': 0.4827608147808576}
accuracy=0.9709750413894653
{'batch_size': 120.0, 'dropout_cnn_block_1': 0.4766610761697723, 'dropout_cnn_block_2': 0.3986001800656231, 'dropout_cnn_block_3': 0.45521116554639846, 'dropout_dense_block_1': 0.6719168017001893, 'dropout_dense_block_2': 0.6398834943153436}
accuracy=0.9687075018882751
{'batch_size': 100.0, 'dropout_cnn_block_1': 0.3844544851491682, 'dropout_cnn_block_2': 0.38062796943543864, 'dropout_cnn_block_3': 0.404274695142409, 'dropout_dense_block_1': 0.397615090307035, 'dropout_dense_block_2': 0.5001406907665598}
accuracy=0.978004515171051
{'batch_size': 170.0, 'dropout_cnn_block_1': 0.32328482780135487, 'dropout_cnn_block_2': 0.3822611718866084, 'dropout_cnn_block_3': 0.3027595687459745, 'dropout_dense_block_1': 0.512750411233